# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [64]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [65]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [66]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [67]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [68]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

session.row_factory = pandas_factory

#### Create Keyspace

In [69]:
session.execute("\
    CREATE KEYSPACE IF NOT EXISTS udacity \
    WITH REPLICATION = \
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"\
)

#### Set Keyspace

In [70]:
session.set_keyspace('udacity')

In [71]:
def insert_data(query, session, values, dtypes, file):
    """
    Pipeline to insert data to a Apache Casssandra table from CSV file    
    Args:
        query (str): INSERT statement query
        session (object): Apache Casssandra session
        values (list): list of indicies to pass to INSERT statement
        dtypes (list): list of types to convert `values`
        file (str): CSV file path   
    Returns:
        None
    """
    assert(len(values) ==  len(dtypes))
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, tuple([dtype(line[x]) for x, dtype in zip(values, dtypes)]))

# filename        
file = 'event_datafile_new.csv'

## Query 1: Song Played Session
    Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
    
## Primary Key
    sessionid and iteminsession are unique value to query artist and song infos

### Partition Key
- sessionid

### Cluster Key
- iteminsession


In [72]:
query = "CREATE TABLE IF NOT EXISTS song_played_session"
query = query + "(artist text, song text, length double, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [73]:
query = "INSERT INTO song_played_session (artist, song, length, sessionId, itemInSession)"
query = query + " VALUES (%s, %s, %s, %s, %s)"
values = [0, 9, 5, 8, 3]
dtypes = [str, str, float, int, int]
insert_data(query, session, values, dtypes, file)


In [79]:
query = "SELECT artist, song, length FROM song_played_session WHERE sessionId = 338 AND itemInSession=4"
try:
    rows = session.execute(query)
    
    print(rows._current_rows)
except Exception as e:
    print(e)

      artist                             song    length
0  Faithless  Music Matters (Mark Knight Dub)  495.3073


## Query 2: Song Playlist Session
    Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

## Primary Key
    userid, sessionid and iteminsession can uniquelly identify query info, iteminsession needs to be cluster key in order ot use "ORDER BY" clause 
### Partition Key
- userid
- sessionid

### Cluster Key
- iteminsession

In [82]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(artist text, song text, itemInSession int, firstName text, lastName text, userId int, sessionId int, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [83]:
query = "INSERT INTO song_playlist_session (artist, song, itemInSession, firstName, lastName, userId, sessionId)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
values = [0, 9, 3, 1, 4, 10, 8]
dtypes = [str, str, int, str, str, int, int]
insert_data(query, session, values, dtypes, file)

In [84]:
query = "SELECT artist, song, firstname, lastname FROM song_playlist_session WHERE userId = 10 AND sessionId = 182 ORDER BY iteminsession ASC"
try:
    rows = session.execute(query)
    
    print(rows._current_rows)
except Exception as e:
    print(e)
    

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


## Query 3: User Played Song
    Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
    
## Primary Key
    sang, sessionid can uniquelly identify query info
### Partition Key
- song

### Cluster Key
- sessionid

In [86]:
query = "CREATE TABLE IF NOT EXISTS user_played_song"
query = query + "(song text, firstName text, lastName text, userId int, PRIMARY KEY (song , userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [87]:
query = "INSERT INTO user_played_song (song, firstName, lastName, userId)"
query = query + " VALUES (%s, %s, %s, %s)"
values = [9, 1, 4, 10]
dtypes = [str, str, str, int]
insert_data(query, session, values, dtypes, file)


In [88]:
query = "SELECT firstname, lastname FROM user_played_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
    print(rows._current_rows)
except Exception as e:
    print(e)

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [89]:
session.execute("drop table song_played_session")

In [90]:
session.execute("drop table song_playlist_session")

In [91]:
session.execute("drop table user_played_song")

### Close the session and cluster connection¶

In [62]:
session.shutdown()
cluster.shutdown()